In [1]:
!git clone https://github.com/mymusise/ChatGLM-Tuning.git

fatal: destination path 'ChatGLM-Tuning' already exists and is not an empty directory.


In [2]:
%cd ChatGLM-Tuning

/output/ChatGLM-Tuning


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-h2bsy09g
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-h2bsy09g
  Resolved https://github.com/huggingface/peft.git to commit 445940fb7b5d38390ffb6707e2a989e89fff03b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 54.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 137.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 141.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 125.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 102.0 MB/s eta 0:00:00

# 准备数据
## cover_alpaca2jsonl.py

In [17]:
import json
from tqdm import tqdm


def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}


def main():
    with open("/openbayes/input/input0/datasets/userstory_detail.jsonl") as f:
        examples = list(f)

    with open("/output/train.jsonl", 'w') as f:
        for example in tqdm(examples, desc="formatting.."):
            f.write(json.dumps(format_example(json.loads(example))) + '\n')


if __name__ == "__main__":
    main()

formatting..: 100%|██████████| 3438/3438 [00:00<00:00, 38078.38it/s]


## tokenize_dataset_rows.py

In [3]:
import json
from tqdm import tqdm

import datasets
import transformers


def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True,return_attention_mask=False,
                add_special_tokens=False)
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        return_attention_mask=False,
        add_special_tokens=False)
    input_ids = prompt_ids + [150001, 150004] + target_ids + [150005]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}


def read_jsonl(path, max_seq_length, skip_overlength=False):
    model_name = "/openbayes/input/input1"
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = transformers.AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature


def main():
    dataset = datasets.Dataset.from_generator(
        lambda: read_jsonl("/output/train.jsonl", 384, False)
    )
    dataset.save_to_disk("/output/train")


if __name__ == "__main__":
    main()

Generating train split: 0 examples [00:00, ? examples/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.

100%|██████████| 3438/3438 [00:01<00:00, 1895.22it/s]

Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-5161fd2f24be51ae/0.0.0. Subsequent calls will reuse this data.


Saving the dataset (0/1 shards):   0%|          | 0/3438 [00:00<?, ? examples/s]

# 训练

## 训练前推理

In [19]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = AutoModel.from_pretrained("/openbayes/input/input1", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('tcp'), PosixPath('443'), PosixPath('//10.111.0.1')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//alchemist-experience'), PosixPath('7890')}
  warn(msg)
/usr/loc


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [ ]:
def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

In [25]:
import json

with open("/openbayes/input/input0/datasets/userstory_detail.jsonl") as f:
    examples = list(f)

with torch.no_grad():
    idx = 0
    for example in examples[:5]:
        item = json.loads(example)
        feature = format_example(item)
        input_text = feature['context']
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        inputs = model.prepare_inputs_for_generation(input_ids)
        for k,v in inputs.items():
            if v is not None:
                inputs[k] = v.to("cuda")
        outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
        out = outputs[0].tolist()[input_ids.size()[-1]:]
        answer = tokenizer.decode(out)
        item['infer_answer'] = answer
        print(input_text)
        print(answer)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        idx += 1

Instruction: create user story tasks
Input:  Animation and Comics:Browse and search for animations and comics
Answer: 
1.   browse the internet for animations and Comics
2.  find some that interest you and read about the creators and故事情节
3.  watch or read the animations or Comics that you like
### 1.Answer:
 
用户故事：可以浏览和搜索动画和漫画
作为一个 Animation and Comics 应用的用户
我想要浏览和搜索动画和漫画
以便于我能够找到我喜欢的动画和漫画

AC 1: 用户可以浏览和搜索动画和漫画
假设 用户打开 Animation and Comics 应用
当 用户点击浏览和搜索动画和漫画按钮
于是 用户可以看到动画和漫画的列表，并可以搜索特定的动画和漫画 


Instruction: create user story tasks
Input:  Animation and Comics:Participate in online forums and discussions
Answer: 
Task 1: Participate in online forums and discussions about animation and Comics.

User Story: I am interested in animation and Comics, and I have found an online forums and discussions where I can share my knowledge and experience with others.

Task 2: Write a post on the forum and share your thoughts and opinions about an animation or Comics title.

User Story: I decided to w

## 训练

In [ ]:
!sed -i "s/THUDM\/chatglm-6b/\/openbayes\/input\/input1/" finetune.py
!cat finetune.py
!python finetune.py \
    --dataset_path /output/train \
    --lora_rank 8 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 1 \
    --max_steps 3000 \
    --save_steps 1000 \
    --save_total_limit 2 \
    --learning_rate 1e-4 \
    --fp16 \
    --remove_unused_columns false \
    --logging_steps 50 \
    --output_dir /output/lora

from transformers.integrations import TensorBoardCallback
from torch.utils.tensorboard import SummaryWriter
from transformers import TrainingArguments
from transformers import Trainer, HfArgumentParser
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType
from dataclasses import dataclass, field
import datasets
import os


tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)


@dataclass
class FinetuneArguments:
    dataset_path: str = field(default="data/alpaca")
    model_path: str = field(default="output")
    lora_rank: int = field(default=8)


class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature i

# 训练后推理

In [3]:
from transformers import AutoModel
import torch


torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained("/openbayes/input/input1", trust_remote_code=True, device_map='auto')

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
from peft import get_peft_model, LoraConfig, TaskType

peft_path = "/output/lora/adapter_model.bin"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('tcp'), PosixPath('443'), PosixPath('//10.111.0.1')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//alchemist-experience'), PosixPath('http'), PosixPath('7890')}
  warn(msg)
/usr/loc

## 原厂测试

In [7]:
import json

def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

with open("/openbayes/input/input0/datasets/userstory_detail.jsonl") as f:
    examples = list(f)

with torch.no_grad():
    idx = 0
    for example in examples[:5]:
        item = json.loads(example)
        feature = format_example(item)
        input_text = feature['context']
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        inputs = model.prepare_inputs_for_generation(input_ids)
        for k,v in inputs.items():
            if v is not None:
                inputs[k] = v.to("cuda")
        outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
        out = outputs[0].tolist()[input_ids.size()[-1]:]
        answer = tokenizer.decode(out)
        item['infer_answer'] = answer
        print(input_text)
        print(answer)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        idx += 1

## 任意Prompt推理

In [13]:
def evaluate(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    inputs = model.prepare_inputs_for_generation(input_ids)
    for k,v in inputs.items():
        if v is not None:
            inputs[k] = v.to("cuda")
    outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
    out = outputs[0].tolist()[input_ids.size()[-1]:]
    answer = tokenizer.decode(out)
    item['infer_answer'] = answer
    print(input_text)
    print(answer)

Instruction: create user story tasks
Input: 购买电影票
Answer:

用户故事:可以购买电影票
作为一个电影票购买者
我想在应用中购买电影票
以便于我可以在电影院观看电影

AC 1: 电影票购买者可以在应用中购买电影票
假设 电影票购买者已经登录了应用
当 电影票购买者点击购买电影票按钮
于是 电影票购买者可以查看电影院的可用座位,选择座位,支付电影票,查看电影票的详细信息


In [ ]:
evaluate("Instruction: create user story tasks\nInput: 购买电影票\nAnswer:")